In [107]:
import pandas as pd
data = pd.read_excel('ps1.xlsx')

In [108]:
df = pd.DataFrame(data)
df.isna().sum()

index                  38469
Order ID               38654
Date                   38744
Status                 38751
Fulfilment             38643
Sales Channel          38552
ship-service-level     38959
Style                  38919
SKU                    38705
Category               38875
Size                   38703
ASIN                   38607
Courier Status         43491
Qty                    38778
currency               44284
Amount                 44397
ship-city              38418
ship-state             38736
ship-postal-code       38523
ship-country           38804
promotion-ids          73106
B2B                    38676
fulfilled-by          101373
Unnamed: 22            72925
dtype: int64

In [109]:
df.drop('index', axis=1, inplace=True)

In [110]:
df['Date'] = df['Date'].backfill()
df['Fulfilment'] = df['Fulfilment'].backfill()
df['ship-service-level'] = df['ship-service-level'].backfill()
df['currency'] = df['currency'].fillna('INR')
df['Size'] = df['Size'].fillna(df['Size'].mode()[0])
df['Sales Channel '] = df['Sales Channel '].fillna('Amazon.in')
df['ASIN'] = df['ASIN'].fillna(0)
df['Category'] = df['Category'].ffill()

C:\Users\tejas\AppData\Local\Temp\ipykernel_12748\1996586410.py:1: FutureWarning: DataFrame.backfill/Series.backfill is deprecated. Use DataFrame.bfill/Series.bfill instead
  df['Date'] = df['Date'].backfill()
C:\Users\tejas\AppData\Local\Temp\ipykernel_12748\1996586410.py:2: FutureWarning: DataFrame.backfill/Series.backfill is deprecated. Use DataFrame.bfill/Series.bfill instead
  df['Fulfilment'] = df['Fulfilment'].backfill()
C:\Users\tejas\AppData\Local\Temp\ipykernel_12748\1996586410.py:3: FutureWarning: DataFrame.backfill/Series.backfill is deprecated. Use DataFrame.bfill/Series.bfill instead
  df['ship-service-level'] = df['ship-service-level'].backfill()


In [111]:
df.loc[
    df["Status"].isna() & (
        (df["Qty"] == 1) | 
        (df["Courier Status"] == "Shipped") | 
        (df["Amount"].notna())
    ), 
    "Status"
] = "Shipped"

df['Status'] = df['Status'].fillna("Cancelled")

In [112]:
df.dropna(subset=['Order ID'], inplace=True)
df.dropna(subset=['SKU'], inplace=True)
df['Amount'] = df['Amount'].fillna(df.groupby('Category')['Amount'].transform('median'))
df['Courier Status'] = df['Courier Status'].fillna(df['Status'])
df.loc[df['Status'] != 'Cancelled', 'Qty'] = df['Qty'].fillna(0)
df['Qty'] = df['Qty'].fillna(1)
df.isna().sum()

Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                 19106
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status            0
Qty                       0
currency                  0
Amount                    0
ship-city             18774
ship-state            18979
ship-postal-code      18869
ship-country          19109
promotion-ids         35935
B2B                   18992
fulfilled-by          49844
Unnamed: 22           35892
dtype: int64

In [113]:
df2 = df
df2['Date'] = pd.to_datetime(df2['Date'], format='%m-%d-%y')
df2['Year-Month'] = df2['Date'].dt.to_period('M')
monthly_sales = df2.groupby('Year-Month')['Amount'].sum().reset_index()
df2.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22,Year-Month
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,...,406.0,BENGALURU,KARNATAKA,560085.0,IN,NaN,0.0,NaN,NaN,2022-04
3,403-9615377-8133951,2022-04-30,Cancelled,Amazon,Amazon.in,Standard,NaN,J0341-DR-L,kurta,L,...,459.0,PUDUCHERRY,NaN,NaN,IN,NaN,0.0,Easy Ship,NaN,2022-04
5,404-1490984-4578765,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,...,824.0,GHAZIABAD,UTTAR PRADESH,201102.0,IN,NaN,NaN,NaN,NaN,2022-04
6,408-5748499-6859555,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,...,736.0,NaN,CHANDIGARH,160036.0,NaN,NaN,0.0,NaN,NaN,2022-04
9,402-4393761-0311520,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,NaN,JNE3461-KR-XXL,kurta,M,...,363.0,NaN,TAMIL NADU,600041.0,IN,NaN,NaN,NaN,NaN,2022-04


In [115]:
df.to_excel('monthly.xlsx')